In [27]:
# 모듈 import
from mmcv_custom import runner
from mmcv import Config

from mmdet.utils import get_device
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [30]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/detectors/detectors_htc_r101_20e_coco.py')

root='../../dataset/'
kfold_root = '../../dataset/stratified_v3/'

# dataset config 수정


cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.ann_file = kfold_root + 'train_fold_1.json' # train json 정보
#cfg.data.train.pipeline[2]['img_scale'] = (1024,1024) # Resize

#cfg.data.test.classes = classes
#cfg.data.test.img_prefix = root
#cfg.data.test.ann_file = root + 'test.json' # test json 정보
#cfg.data.test.ann_file = kfold_root + 'cv_val_1.json' # test json 정보
#cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
#cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.val.ann_file = kfold_root + 'val_fold_1.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize



cfg.data.samples_per_gpu = 2


cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/htc_detectors_resnext101'

cfg.model.bbox_head.num_classes = 10
#cfg.model.roi_head.bbox_head[0].num_classes = 10
#cfg.model.roi_head.bbox_head[1].num_classes = 10
#cfg.model.roi_head.bbox_head[2].num_classes = 10


#cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=5, interval=2)
cfg.device = get_device()

# cfg.runner.max_epochs = 36
#cfg.lr_config = dict(policy='CosineAnnealing', warmup='linear', warmup_iters=1000, warmup_ratio=1.0 / 10, min_lr_ratio=1e-5)

meta = dict()
meta['fp16'] = dict(loss_scale=dict(init_scale=512))
cfg.runner.meta = meta
#cfg.optimizer.lr=0.001


log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(project='level2-objectdetection', name='detectors_htc_r101_20e_coco_base', entity='hi-ai')
            )])

AttributeError: 'ConfigDict' object has no attribute 'bbox_head'

In [ ]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


In [ ]:

# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2023-05-09 15:14:11,237 - mmdet - INFO - load checkpoint from torchvision path: torchvision://resnet101
2023-05-09 15:14:11,720 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset_l.weight, layer2.1.conv2.offset_l.bias, layer2.2.c

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True,)

2023-05-09 15:14:26,828 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-09 15:14:26,878 - mmdet - INFO - Start running, host: root@548bdb2194e9, work_dir: /opt/ml/baseline/mmdetection/work_dirs/htc_detectors_resnext101
2023-05-09 15:14:26,879 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


AssertionError: The `num_classes` (80) in Shared2FCBBoxHead of MMDataParallel does not matches the length of `CLASSES` 10) in CocoDataset